In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_json("../data/cocktail_dataset.json")
data.set_index('id', inplace=True)

In [3]:
data.columns

Index(['name', 'category', 'glass', 'tags', 'instructions', 'imageUrl',
       'alcoholic', 'createdAt', 'updatedAt', 'ingredients'],
      dtype='object')

In [4]:
print(data.isnull().sum())

name             0
category         0
glass            0
tags            99
instructions     0
imageUrl         0
alcoholic        0
createdAt        0
updatedAt        0
ingredients      0
dtype: int64


In [5]:
data['tags'] = data['tags'].apply(lambda x: x if isinstance(x, list) else [])

Zaczynam od usunięcia imageUrl, createdAt, updatedAt, ponieważ nie wnoszą nic, a same daty dotyczą jednego dnia

In [6]:
data = data.drop(columns=["imageUrl", "createdAt", "updatedAt"])

In [7]:
# oficjalne IBA
with open("../data/official_IBA.txt", "r", encoding="utf-8") as file:
    official_IBA = file.read().splitlines()

# tworzę nową kolumnę z wartościami 0 i 1 dla tagów IBA
data["IBA"] = 0
for i, row in data.iterrows():
    if row["name"] in official_IBA:
        data.at[i, "IBA"] = 1

In [8]:
data.head()

,name,category,glass,tags,instructions,alcoholic,ingredients,IBA
id,,,,,,,,
11000,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",Muddle mint leaves with sugar and lime juice. ...,1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1
11001,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",Place sugar cube in old fashioned glass and sa...,1,"[{'id': 513, 'name': 'Water', 'description': '...",1
11002,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",Combine all ingredients (except cola) and pour...,1,"[{'id': 305, 'name': 'Light Rum', 'description...",0
11003,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic]","Stir into glass over ice, garnish and serve.",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1
11004,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]","Shake with ice. Strain into chilled glass, gar...",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1


Dodaję każdemu wierszowi tag Alcoholic, zgodny z kolumną alcoholic

In [9]:
for i, row in data.iterrows():
    if type(row["tags"]) == list:
        if "Alcoholic" not in row["tags"]:
            row["tags"].append("Alcoholic")
    else:
        row["tags"] = ["Alcoholic"]

In [10]:
data.head()

,name,category,glass,tags,instructions,alcoholic,ingredients,IBA
id,,,,,,,,
11000,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",Muddle mint leaves with sugar and lime juice. ...,1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1
11001,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",Place sugar cube in old fashioned glass and sa...,1,"[{'id': 513, 'name': 'Water', 'description': '...",1
11002,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",Combine all ingredients (except cola) and pour...,1,"[{'id': 305, 'name': 'Light Rum', 'description...",0
11003,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic]","Stir into glass over ice, garnish and serve.",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1
11004,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]","Shake with ice. Strain into chilled glass, gar...",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1


In [11]:
set(data.category)

{'Cocktail', 'Ordinary Drink', 'Punch / Party Drink'}

Są 3 kategorie drinków, więc zastosuję one-hot encoding

In [12]:
data = pd.get_dummies(data, columns=['category'])

dummy_columns = [col for col in data.columns if col.startswith('category_')]
data[dummy_columns] = data[dummy_columns].astype(int)

One-hot encoding zastosuję również na kolumnie glass

In [13]:
set(data.glass)

{'Brandy snifter',
 'Champagne flute',
 'Cocktail glass',
 'Collins glass',
 'Copper Mug',
 'Highball glass',
 'Old-fashioned glass',
 'Pousse cafe glass',
 'Whiskey Glass',
 'Whiskey sour glass',
 'White wine glass'}

In [14]:
data = pd.get_dummies(data, columns=['glass'])

dummy_columns = [col for col in data.columns if col.startswith('glass')]
data[dummy_columns] = data[dummy_columns].astype(int)

Na bazię kolumny name, dodam kolumnę ingredientInName. W EDA okazało się, że 25% drinków zawiera jakiś składnik w nazwię.

In [15]:
data["ingredientInName"] = 0

for i, row in data.iterrows():
    for ingredient in row["ingredients"]:
        if ingredient['name'] in row["name"]:
            data.at[i, "ingredientInName"] = 1

data.head()
data["ingredientInName"].value_counts()

ingredientInName
0    105
1     29
Name: count, dtype: int64

Teraz usuwam kolumne name, da się ją odzyskać po id.

In [16]:
data = data.drop(columns=["name"])

Dodaję kolumne instructionsLength określającą długość instrukcji w liczbie słów, dodatkowo skaluję tę kolumne używając MinMaxScaler().
Usuwam kolumne instructions.

In [17]:
data['instructionsLength'] = data['instructions'].apply(lambda x: len(x.split()))

scaler = MinMaxScaler()
data['instructionsLength'] = scaler.fit_transform(data[['instructionsLength']])

data = data.drop(columns=["instructions"])

Dodaję kolejno ilość tagów tagsCount i ingredientsCount, po czym je skaluję.

In [18]:
data['tagsCount'] = data['tags'].apply(lambda x: len(x) if x is not None else 0)
data['ingredientsCount'] = data['ingredients'].apply(lambda x: len(x) if x is not None else 0)

scaler = MinMaxScaler()

data[['tagsCount', 'ingredientsCount']] = scaler.fit_transform(data[['tagsCount', 'ingredientsCount']])


In [19]:
data.head()

,tags,alcoholic,ingredients,IBA,category_Cocktail,category_Ordinary Drink,category_Punch / Party Drink,glass_Brandy snifter,glass_Champagne flute,glass_Cocktail glass,glass_Collins glass,glass_Copper Mug,glass_Highball glass,glass_Old-fashioned glass,glass_Pousse cafe glass,glass_Whiskey Glass,glass_Whiskey sour glass,glass_White wine glass,ingredientInName,instructionsLength,tagsCount,ingredientsCount
id,,,,,,,,,,,,,,,,,,,,,,
11000,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.509804,1.000000,0.75
11001,"[IBA, Classic, Alcoholic, Expensive, Savory]",1,"[{'id': 513, 'name': 'Water', 'description': '...",1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.568627,0.444444,0.50
11002,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",1,"[{'id': 305, 'name': 'Light Rum', 'description...",0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.392157,0.666667,1.00
11003,"[IBA, Classic, Alcoholic]",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.222222,0.25
11004,"[IBA, Classic, Alcoholic, ContemporaryClassic]",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.294118,0.333333,0.50


Dodam kolumne classic zawierającą drinki z tagami ContemporaryClassic i Classic   

In [20]:
data["classic"] = 0
for i, row in data.iterrows():
    if "ContemporaryClassic" in row["tags"] or "Classic" in row["tags"]:
        data.at[i, "classic"] = 1

Dodaję kolumne gin

In [21]:
data["gin"] = 0
for i, row in data.iterrows():
    for ingredient in row["ingredients"]:
        if ingredient['name'] == "Gin":
            data.at[i, "gin"] = 1

In [22]:
data.head()

,tags,alcoholic,ingredients,IBA,category_Cocktail,category_Ordinary Drink,category_Punch / Party Drink,glass_Brandy snifter,glass_Champagne flute,glass_Cocktail glass,glass_Collins glass,glass_Copper Mug,glass_Highball glass,glass_Old-fashioned glass,glass_Pousse cafe glass,glass_Whiskey Glass,glass_Whiskey sour glass,glass_White wine glass,ingredientInName,instructionsLength,tagsCount,ingredientsCount,classic,gin
id,,,,,,,,,,,,,,,,,,,,,,,,
11000,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.509804,1.000000,0.75,1,0
11001,"[IBA, Classic, Alcoholic, Expensive, Savory]",1,"[{'id': 513, 'name': 'Water', 'description': '...",1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.568627,0.444444,0.50,1,0
11002,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",1,"[{'id': 305, 'name': 'Light Rum', 'description...",0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.392157,0.666667,1.00,0,1
11003,"[IBA, Classic, Alcoholic]",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.222222,0.25,1,1
11004,"[IBA, Classic, Alcoholic, ContemporaryClassic]",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.294118,0.333333,0.50,1,0


Chociaż dałoby się wycisnąć z nich więcej, usuwam tagi i składniki.

In [23]:
data = data.drop(columns=["tags", "ingredients"])

In [24]:
data.head()

,alcoholic,IBA,category_Cocktail,category_Ordinary Drink,category_Punch / Party Drink,glass_Brandy snifter,glass_Champagne flute,glass_Cocktail glass,glass_Collins glass,glass_Copper Mug,glass_Highball glass,glass_Old-fashioned glass,glass_Pousse cafe glass,glass_Whiskey Glass,glass_Whiskey sour glass,glass_White wine glass,ingredientInName,instructionsLength,tagsCount,ingredientsCount,classic,gin
id,,,,,,,,,,,,,,,,,,,,,,
11000,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.509804,1.000000,0.75,1,0
11001,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.568627,0.444444,0.50,1,0
11002,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.392157,0.666667,1.00,0,1
11003,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.222222,0.25,1,1
11004,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.294118,0.333333,0.50,1,0


In [25]:
# zapisuję dane do pliku

data.to_csv("../data/cocktail_dataset_preprocessed.csv")